# ETL for images of Google Art & Culture

This notebooks present a code for extract images of Google Art & Culture.

**Extraction**
* Extract init database from `data_unpacked.csv`, this is obtained in the first phase of web scraping
* Extract images through multithreading request

**Transform** 
* Transform `data_unpacked.csv` keeping only `link` column
* Create a `index` column, and `is_download` for check if an image is downloaded or not

**Load**
* Save retrival information into csv file

In [43]:
import sys
sys.path.append('../src/')
from Parallel import parallel
import pandas as pd
from Database import Database, CUSTOMER
import requests
import time

def download(data):
    id = data[0]
    link = data[1]
    path = f'../data/raw/img/{id}.jpg'
    img = requests.get(link).content
    with open(path, 'wb') as handler:
        handler.write(img)
    return (True,f'{id}.jpg')

def extraction():
    return pd.read_excel('../data/processed/data_unpacked.xlsx')

def transform(df):
   
    t_init = time.process_time()
    '''
    Get interest attributes 
    '''
    df = df[['url']]
    df['index'] = df.index
    df = df[['index','url']]   

    '''
    Convert data into tuples
    '''
    data = list(df.itertuples(index=False, name=None))

    '''
    Apply parallel transformation for download files 
    '''
    data_processed = parallel(download,data)

    '''
    Convert raw data_processed into dataframe
    '''

    df_processed = pd.DataFrame(list(data_processed), columns=['is_download','filename'])

    '''
    Combining original attributes with new attributes
    '''
    df_transform = pd.concat([df, df_processed], axis=1)


    '''
    Check the time of extraction process
    '''
    t_end=  time.process_time()
    print('Time passed (seg):',t_end-t_init)
    return df_transform

def load(df):
    df.to_csv('..\data\processed\links.csv', index = False) 




In [44]:
#ETL Process
if __name__ == '__main__':
    df_extraction = extraction()
    df_transform = transform(df_extraction)
    load(df_transform)



C:\Users\user\AppData\Local\Temp\ipykernel_14424\3133619699.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['index'] = df.index
C:\Users\user\AppData\Local\Temp\ipykernel_14424\3133619699.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_download'] = False


Running
Time passed (seg): 148.421875
